In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import re
import monpa

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


In [3]:
monpa.load_userdict("../../setting/MONPA_斷詞字典.txt")

In [3]:
df = pd.read_json("../backup/vector_all.json")
# 使用drop()方法删除指定的列
df.drop(columns=['vector_content', 'vector', 'vector_title',
        'vector_tag', 'vector_tf'], inplace=True)
# inplace=True表示在原地修改DataFrame，如果不指定inplace=True，则需要将结果赋值给一个新的DataFrame变量。

In [6]:
df

,class,title,tag,TF,url,content,train_word
0,生活科技,種瓜得瓜 AI也略懂略懂,虛假相關 因果關係 手寫數字 大數據與資料探勘 不變性 方法 MNIST 人工智慧,資料 數字 Bottou AI 數據 顏色,https://highscope.ch.ntu.edu.tw/wordpress/?p=8...,方法 因果關係 大數據與資料探勘 種瓜得瓜 AI也略懂略懂 手寫數字 參考資料 L. Bot...,種瓜得瓜 AI也略懂略懂生活科技資料 數字 Bottou AI 數據 顏色虛假相關 因果關係...
1,生活科技,眼見不為憑：假新聞攻防戰,對抗生成網路 媒體識讀 倫理 人工智慧 稀疏編碼 Deepfake 假新聞 Sparse c...,影片 Deepfake 影像 資料 AI 假消息,https://highscope.ch.ntu.edu.tw/wordpress/?p=8...,參考資料 「剛開始看覺得沒什麼問題，但仔細看之後會覺得不太對，但也說不出奇怪的地方在哪。」這...,眼見不為憑：假新聞攻防戰生活科技影片 Deepfake 影像 資料 AI 假消息對抗生成網路...
2,生活科技,演化，AI的第三條路！(1/2),基因 機器學習 PID 演化 調諧規則 方法 人工智慧 遺傳編程,遺傳 函數 規劃 電路 控制器 演化,https://highscope.ch.ntu.edu.tw/wordpress/?p=8...,電腦程式的演化，是個比數學函數複雜的例子。譬如利用迭代法及記憶方式為蛋白質序列進行分類的電腦...,演化，AI的第三條路！(1/2)生活科技遺傳 函數 規劃 電路 控制器 演化基因 機器學習 ...
3,生活科技,理解與預測：科學的兩條路,人工智慧 康德 中文房間 錯覺 知識論 應用 本體論 悖論 科學研究的變革,一個 預測 理解 科學 人類 之間,https://highscope.ch.ntu.edu.tw/wordpress/?p=8...,「教」讓我們能交流所知的因果關係（如果按照這些規則，那麼你就可以完成除法）；「學」則讓我們串...,理解與預測：科學的兩條路生活科技一個 預測 理解 科學 人類 之間人工智慧 康德 中文房間 ...
4,生活科技,別中了社群媒體的圈套(2/2),打卡 倫理 多巴胺 科技成癮與依賴 按讚 人工智慧 成癮,多巴胺 媒體 社群 手機 釋放 使用者,https://highscope.ch.ntu.edu.tw/wordpress/?p=8...,我們是否會像酒精跟藥物成癮那樣對手機「成癮」，這問題頗有爭議，不過無論是酒精、藥物還是手機，...,別中了社群媒體的圈套(2/2)生活科技多巴胺 媒體 社群 手機 釋放 使用者打卡 倫理 多巴...
...,...,...,...,...,...,...,...
25223,環境能源,利比亞重奪油港、出口恐增，油價再添壓力,None,利比亞 OPEC 14 萬桶 減產,https://finance.technews.tw/2017/03/15/rising-...,石油輸出國組織（OPEC）減產出現疑慮，美國頁岩油又拚命增產，油市信心潰散，價格連日走低。就...,利比亞重奪油港、出口恐增，油價再添壓力環境能源利比亞 OPEC 14 萬桶 減產 石油輸出國...
25224,環境能源,解決缺電雙管齊下，經濟部：核二廠備而不用,None,核二廠 供電 號機 格架案 報告,https://technews.tw/2017/04/13/kuosheng-nuclea...,核二廠 1 號機格架案動工，增加燃料棒裝載池空間，時代力量黨團要求應向立院報告。經濟部次長沈...,解決缺電雙管齊下，經濟部：核二廠備而不用環境能源核二廠 供電 號機 格架案 報告 核二廠 1...
25225,環境能源,住宅太陽能夯！SunPower 推用戶貸款案 股價創 09 年高,None,太陽能 貸款 SunPower 計畫 安裝,https://finance.technews.tw/2014/06/17/%e4%bd%...,全美第二大太陽能面板製造商SunPower Corp.在美國股市開盤前發布新聞稿宣佈，該公司...,住宅太陽能夯！SunPower 推用戶貸款案 股價創 09 年高環境能源太陽能 貸款 Sun...
25226,環境能源,矽晶圓和電池報價破底，太陽能產業現陰霾,None,太陽能 報價 中國 矽晶圓 amp,https://technews.tw/2016/07/04/silicon-wafer-b...,隨著中國太陽能搶裝潮結束，太陽能景氣又向下探底，矽晶圓和電池報價同步下殺，近日一般轉換效率的...,矽晶圓和電池報價破底，太陽能產業現陰霾環境能源太陽能 報價 中國 矽晶圓 amp 隨著中國太...


In [4]:
# 讀取文字檔並轉換成串列
def txt_to_list(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.readlines()
            content = [line.strip() for line in content]
        return content
    except FileNotFoundError:
        print("找不到指定的檔案。請檢查檔案路徑是否正確。")
        return []
    except Exception as e:
        print("讀取檔案時發生錯誤：", e)
        return []


# 指定文字檔路徑
stop_word_list = txt_to_list("../../setting/stopwords_chinese.txt")
# stop_word_list

In [11]:
def cutSentence(sentence):
    seg = []
    for item in sentence.split("，"):
        if item != "\n":
            seg.extend(monpa.cut(str(item+"，")))

    word_list = seg[:-1]

    word_list = [word for word in word_list if word not in stop_word_list]
    # 将词列表拼接成句子
    sentence = ' '.join(word_list)
    return sentence

In [9]:
df["cuted"] = df["content"].apply(lambda x: cutSentence(x))

In [10]:
df.to_json('../backup/99. 斷詞後_df.json')

In [5]:
dfi = pd.read_json("../backup/99. 斷詞後_df.json")

In [28]:
import re
from collections import Counter


def calculate_word_frequency(row, text):
    words = re.findall(r'\b\w+\b', text)  # 利用正則表達式分割文本成單詞
    word_freq = Counter(words)  # 計算字詞的字頻
    total_words = len(words)  # 總字詞數

    # 計算每個字的tf
    word_tf = {word: word_freq[word] / total_words for word in word_freq}

    # 將結果轉換成指定的格式
    result = [{"article": row, "word": word, "count": word_freq[word],
               "tf": word_tf[word]} for word in word_freq if word_tf[word] > 0.003]
    df_array.extend(result)

In [32]:
# 提取字頻和tf
corpus = cutSentence("""
    四年後，虎克被任命為新創辦皇家科學會實驗室的組長，負責每星期所討論的實驗工作。1665 年，他終於受聘為Gresham學院的幾何學教授，是英國第一位有給職的研究科學家。1661-1664 年間，他利用顯微鏡進行了一系列的觀察與實驗，將所得到的結果寫成了《微物圖解》。 虎克誕生於威特小島的一個牧師家庭，他起初想當藝術家，因此 13 歲時就到當時最有名的肖像畫家處當學徒，在那裡他抱怨油彩和油漆讓他的胸口不舒服，所以就離開學徒生涯到西敏寺學校去上學。在學校裏，他精通了古文，學會彈奏風琴，利用飛行物來做實驗，還據說只花一星期就精通了希臘數學家歐基理得的書《幾何原本》(Elements)中的前六卷。1653 年，虎克在牛津的基督教堂當聖歌隊員，遇到了後來創辦皇家科學會(the Royal Society)的人。1658 年時，他去當波義耳(Robert Boyle)的助手，應用他的機械技巧來改良抽氣機，在化學與實際的實驗方面學到了純熟完美的技術。 《微物圖解》於 1665 年 1 月首度在書店上架後立刻造成轟動，它不僅提供給科學家嶄新的資料寶藏，而且對實驗科學提供了既清晰又完美的例證。例如，由顯微鏡底下所簡單觀察到的一片木炭即可認識到細胞的存在；描述一隻蒼蠅的構造就可寫成空氣動力學、聲學、與波動型態等方面的實驗論文。事實上，在《微物圖解》中所記錄的 60 個觀察報告，加上 58 幅繪製精美的版畫，每一個都那麼詳細，足以提供物理上做更深入的研究。虎克在藝術方面的才華，使他能忠實地解釋、描述 1660 年代複式顯微鏡中所呈現的粗糙影像，對於此書的完成與出版助益很大。 物理 在古典科學的書籍中，幾乎沒有一本能夠像牛頓的《自然哲學的數學原理》(Principia)和達爾文的《物種起源》(Origin of the Species)一般，擁有彌久不衰的影響力，但有一本書卻經常為大家所遺忘，那就是《微物圖解》(Micrographia)。《微物圖解》的作者是虎克(Robert Hooke，1635-1703，於 1670 年代提出虎克定律)，雖然他的知名度無法和他同時期的牛頓相比，可是他卻是英國 17 世紀最聰明、最多才多藝的科學家之一。《微物圖解》除了在科學上的重要性有目共睹外，它更打開了原來看不見的宇宙，將其呈現給一般的讀者。 物理學發展史 虎克晚年時健康惡化，心血管毛病加上糖尿病使得他的雙腿腫脹、胸口疼痛、暈眩、虛弱、全盲，終於在 1703 年 3 月 3 日過世。雖然他在世人的眼中看來很成功，但是他的名譽卻因他和其他科學家的紛爭而受損，其中包括和惠更斯對彈簧調整器的意見不同，和牛頓先是光學問題，後又就萬有引力的平方反比法則提出的先後爭執。這些事情總是深深地刺傷他的自尊心，所以他後來變得易怒、鬱悶、多疑，總覺得被牛頓出賣了，在社會上得不到應有的聲望。 物理學發展史 雖然虎克從未獲得世人如對牛頓般的推崇，然而，他的確有著特殊的創造力，多才多藝，對古文的造詣深厚，有著高度的繪圖技巧與實際的手藝技能。最重要的，根據牛津大學歷史學家 Alan Chapman 所說：「他是證明『實驗哲學』真的有效，可用以拓展自然知識領域的人。他是歐洲最後一個文藝復興的人，是英國的『達文西』(Leonardo)」。〈譯者按：達文西是義大利文藝復興三大藝術巨匠之一，多才多藝，代表作有蒙娜麗莎、最後的晚餐等。〉 虎克後來在光學與重力方面的研究雖然因牛頓的貢獻而相形失色，但是在當時，他在科學儀器的發明與設計方面是無可匹敵的。例如，他發明了錶的發條來控制齒輪的平衡、複式顯微鏡、輪式氣壓計、和目前全球機動車所使用的接合器：虎克接合器。他率先強調解決天文儀器解析度的重要性，製作了第一個反射望遠鏡，並用它來觀測火星的自轉，也讓他最早察覺到一個雙星系統。此外，他也是一個高超的建築師，設計了許多倫敦的建築物，雖然現在只剩幾幢而已。 這本書對外行人來說也有相同的震撼效果，主要是因為它易讀的寫作方式和豐富的圖解內容，讓人對於平凡、熟悉的事物有了深刻的新視野：一個細小的針頭看起來像一根粗糙的紅蘿蔔；纖細的絲很像籃網；熄滅的火花看似一堆木炭。Samuel Pepys 說他仔細地閱讀此書，直到深夜 2 點，並宣稱：「這是我一生中所讀過最精巧、有創意的一本書」。Peps 後來買了自己的科學儀器去加入皇家科學會，終於在 1684 年當上科學會的會長。
""")

In [34]:
output = calculate_word_frequency(0, corpus)

In [33]:
df_array = []

In [35]:
df_array

[{'article': 0, 'word': '年', 'count': 8, 'tf': 0.016597510373443983},
 {'article': 0, 'word': '虎克', 'count': 9, 'tf': 0.01867219917012448},
 {'article': 0, 'word': '新', 'count': 2, 'tf': 0.004149377593360996},
 {'article': 0, 'word': '創辦', 'count': 2, 'tf': 0.004149377593360996},
 {'article': 0, 'word': '皇家科學會', 'count': 2, 'tf': 0.004149377593360996},
 {'article': 0, 'word': '實驗', 'count': 7, 'tf': 0.014522821576763486},
 {'article': 0, 'word': '1665', 'count': 2, 'tf': 0.004149377593360996},
 {'article': 0, 'word': '終於', 'count': 3, 'tf': 0.006224066390041493},
 {'article': 0, 'word': '幾何', 'count': 2, 'tf': 0.004149377593360996},
 {'article': 0, 'word': '學', 'count': 2, 'tf': 0.004149377593360996},
 {'article': 0, 'word': '英國', 'count': 3, 'tf': 0.006224066390041493},
 {'article': 0, 'word': '第一', 'count': 2, 'tf': 0.004149377593360996},
 {'article': 0, 'word': '研究', 'count': 3, 'tf': 0.006224066390041493},
 {'article': 0, 'word': '科學家', 'count': 4, 'tf': 0.008298755186721992},
 {'a

In [52]:
dfi.apply(lambda row: calculate_word_frequency(row.name, row["cuted"]), axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
25223    None
25224    None
25225    None
25226    None
25227    None
Length: 25228, dtype: object

In [6]:
df = dfi.iloc[0:2].copy()

In [56]:
dfr = pd.DataFrame(df_array)

In [67]:
dfr

,article,word,count,tf
0,0,方法,2,0.005900
1,0,因果,2,0.005900
2,0,關係,2,0.005900
3,0,大數據,1,0.002950
4,0,資料,11,0.032448
...,...,...,...,...
5915065,25227,成本,1,0.003610
5915066,25227,未來,1,0.003610
5915067,25227,降,1,0.003610
5915068,25227,6,1,0.003610


In [68]:
import pandas as pd

# 假设你已经有了名为df的DataFrame，其中包含article、word、count和tf四个列

# 按article进行分组
grouped_df = dfr.groupby('article')

# 定义一个函数，用于获取前10大TF的记录


def top_10_tf(group):
    return group.nlargest(10, 'tf')


# 应用函数获取每个group的前10大TF
top_10_tf_df = grouped_df.apply(top_10_tf).reset_index(drop=True)

# 打印结果
print(top_10_tf_df[['article', 'word', 'count', 'tf']])

        article    word  count        tf
0             0      資料     11  0.032448
1             0      數字      8  0.023599
2             0  Bottou      7  0.020649
3             0      虛假      6  0.017699
4             0      系統      6  0.017699
...         ...     ...    ...       ...
252182    25227       儲      4  0.014440
252183    25227       綠      3  0.010830
252184    25227      電廠      3  0.010830
252185    25227      10      3  0.010830
252186    25227      能源      3  0.010830

[252187 rows x 4 columns]


In [91]:
top_10_tf_df.to_json('99. 文本數與tf_df.json')

In [92]:
word_count_df = pd.read_json("99. 文本數與tf_df.json")

In [99]:
import pandas as pd

# 假设你已经有了名为word_count_df的DataFrame，其中包含word列和其他列

# 使用drop_duplicates方法获取word列的不重複值
unique_words_df = word_count_df[['word']].drop_duplicates()

          word
0           資料
1           數字
2       Bottou
3           虛假
4           系統
...        ...
252094     茨城縣
252109      扶持
252114    阿思拜克
252139     利比亞
252176      下殺

[28373 rows x 1 columns]


In [ ]:
unique_words_df["vector"] = unique_words_df["word"].apply(
    lambda x: sbert_model.encode(x).tolist())

In [88]:
index = 6890
top_10_tf_df.iloc[10 * index:10 * index + 10, :]

,article,word,count,tf
68900,6890,湖泊,8,0.021108
68901,6890,生物,8,0.021108
68902,6890,藍牙,8,0.021108
68903,6890,無線,6,0.015831
68904,6890,中,5,0.013193
68905,6890,時,5,0.013193
68906,6890,詢問,5,0.013193
68907,6890,系統,5,0.013193
68908,6891,自閉症,18,0.040816
68909,6891,基因,16,0.036281


In [103]:
# Import modules
from sentence_transformers import SentenceTransformer, util
sbert_model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

c:\Users\vm\Anaconda3\envs\liang_trello_finder_tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
